# New topo DEMs in Southwest WA
## Some Observations and Questions

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

We have been investigating the new CUDEM tiles published in December, 2025 and have some questions...

This notebook illustrates some of the questions, and uses the list of topo sources found here:

https://depts.washington.edu/ptha/CHTuser/topo/topo-sources-cascadia/

This is a possibly incomplete list of places where we have found the topo DEMs for the Cascadia region. Corrections or additions welcome!

In many cases the most recent tiles are available in two different forms:

- A tif file referenced to NAVD88 e.g. in the catalog
  
  https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/index.html
  
- A netCDF file referenced to MHW (on the THREDDS server in folders for `nthmp`), e.g. in the catalog

  https://www.ngdc.noaa.gov/thredds/catalog/tiles/nthmp/tiled_19as/catalog.html

Many of our questions pertain to the difference between these:

1. Why are different formats (tif/nc) used for the different versions?
2. When reading these files, it seems that the `x,y` values in the .nc files are offset by integer multiples of $\Delta = $ 1/9" from integer values of longitude, latitude.  By contrast, the `x,y` in the .tif files are offset by half-integral offsets.  In the .nc files the first and last latitudes are exactly at 47.00 and 47.25, for example, whereas in the .tif files there are always a few points outside the tile with `y` values at $47 \pm \Delta/2$, for example.  Why are these indexed differently?
3. Does this mean that one set of values should be viewed as pointwise values and the other as cell average values?  Running `gdalinfo` on a .tif file shows that it includes the metadata

     AREA_OR_POINT=Area
   
   But the .nc file does not seem to have this metadata field.

   The elevations `z` in the two files are not identical (even after accounting for the difference
   from NAVD88 to MHW), so they are not the same values simply offset by $\Delta/2$.
   
5. How was the conversion done from .tif to .nc?  Which of these is considered more accurate, and the one that we should use if possible?  Presumably the .tif files, which are referenced to NAVD88?


6. I assume the conversion from NAVD88 to MHW was done with VDATUM?  Was this applied to the entire .tif file? (The online version seems to only allow ASCII files or single points as input. The ASCII file version does not seem to be working and the single point version doesn't return values very far inland.)

5. In at least one case, `n47x00_w124x25`, the .nc file is labelled `2025v1` while the .tif file is labelled `2025v2`. Is the .nc file obsolete?  Are the differences from one version to another described anywhere?


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd

In [ ]:
import find_topo_source

## Select a set of tiles

Specify the NW corner of the 0.25 x 0.25 degree tiles to search for:

In [ ]:
eighth = 1/8  # half tile width, 0.0125 degrees
yrange = arange(46.5, 47.25+eighth, 0.25)
xrange = arange(-124.25, -123.5+eighth, 0.25)

if 1:
    name = 'NCEI tiles in SW Washington'
    find_topo_source.make_combined_tile_kml(name, xrange, yrange)
    
# make list of tile names (north to south):
tile_names = []
for y in yrange[-1::-1]:
    for x in xrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        #print(f'midpoints: {xm:.3f}, {ym:.3f}, NW corner:  {x:.3f}, {y:.3f}, tile: {tile_name}')
        tile_names.append(tile_name)

## Search for tiles in catlogs:

In [ ]:
tif_2024 = []
tif_2025 = []
nc_2025 = []
for tile_name in tile_names:
    tile_urls = find_topo_source.find_tile_url(tile_name, verbose=False)
    for url in tile_urls:
        if url[-3:]=='.nc' and '2025' in url:
            nc_2025.append(tile_name)
        if url[-4:]=='.tif' and '2025' in url:
            tif_2025.append(tile_name)
        if url[-4:]=='.tif' and '2024' in url:
            tif_2024.append(tile_name)

print('Found these versions:\n  nc_2025 = ',nc_2025)
print('\n  tif_2025 = ',tif_2025)
print('\n  tif_2024 = ',tif_2024)

## Map of tiles

In [ ]:
m = folium.Map(location=(46.75,-124), zoom_start=9, height=800, scrollWheelZoom=False)

#gdf = gpd.read_file('NCEI_tiles_in_SW_Washington.kml', driver='KML')
#folium.GeoJson(gdf).add_to(m)

for y in yrange[-1::-1]:
    for x in xrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        folium.Marker(
            location=[y-eighth,x+eighth],
            popup = f"<b>Tile:</b>\n {tile_name}",
            tooltip = "Click for info",
            #tooltip = f"<b>Tile:</b>\n {tile_name}",
            icon=folium.Icon(color="red", icon="info-sign") # Customize the marker's appearance
        ).add_to(m) 
        x1 = x
        x2 = x + 0.25
        y1 = y - 0.25
        y2 = y

        if tile_name in nc_2025:
            color = 'green'
            tip = 'nc and tif'
        elif tile_name in tif_2025 + tif_2024:
            color = 'blue'
            tip = 'tif only'
        else:
            color = 'red'
            tip = 'not found'
        
        folium.Polygon(
            locations=[[y1,x1], [y1,x2], [y2,x2], [y2,x1]],
            color="black", # Outline color
            weight=1,
            fill=True,
            fillColor=color,
            fillOpacity=0.2,
            tooltip=tip
        ).add_to(m)

for lat in arange(46.25, 47.3, 0.25):
    folium.PolyLine(
           [[lat,-124.75], [lat,-123.25]],
           color='black', weight=1, opacity=0.5
       ).add_to(m)

    #Add label at edge
    folium.Marker(
        [lat+0.03, -124.7],
        icon=folium.DivIcon(html=f'<div style="font-size: 12pt; color: gray;">{lat}°</div>')
        ).add_to(m)

# save as stand-alone html file:
fname = 'SoWA-Tiles-Map.html'
m.save(fname)
print('Created ',fname)

# display map inline:
m

The stand-alone map can be viewed at [interactive map](SoWA-Tiles-Map.html).

## Versions of these tiles found on NCEI webpages

Print out more details of which catalogs tiles are found in and urls:

In [ ]:

tile_names = []
for y in yrange[-1::-1]:
    print('\n=====================')
    print(f'Latitude y = {y:.2f}:')
    print('=====================')

    for x in xrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        print('\n------------------------------------------------')
        print(f'Tile: {tile_name},  NW corner:  {x:.3f}, {y:.3f}')
        tile_names.append(tile_name)
        tile_urls = find_topo_source.find_tile_url(tile_name, verbose=True)
        if len(tile_urls) == 0:
            print(f'  {tile_name} not found')
